In [4]:
import torch
import numpy as np
import os
import zipfile

# root_path = './../datasets'
# processed_folder =  os.path.join(root_path)

# zip_ref = zipfile.ZipFile(os.path.join(root_path,'omniglot.zip'), 'r')
# zip_ref.extractall(root_path)
# zip_ref.close()

root_dir = './../datasets/omniglot/python'

### 数据预处理

In [5]:
## 数据预处理

# import torchvision.transforms as transforms
# from PIL import Image

# '''
# an example of img_items:
# ( '0709_17.png',
#   'Alphabet_of_the_Magi/character01',
#   './../datasets/omniglot/python/images_background/Alphabet_of_the_Magi/character01')
# '''
# def find_classes(root_dir):
#     img_items = []
#     for (root, dirs, files) in os.walk(root_dir): 
#         for file in files:
#             if (file.endswith("png")):
#                 r = root.split('/')
#                 img_items.append((file, r[-2] + "/" + r[-1], root))
#     print("== Found %d items " % len(img_items))
#     return img_items

# ## 构建一个词典{class:idx}
# def index_classes(items):
#     class_idx = {}
#     count = 0
#     for item in items:
#         if item[1] not in class_idx:
#             class_idx[item[1]] = count
#             count += 1
#     print('== Found {} classes'.format(len(class_idx)))
#     return class_idx
        

# img_items =  find_classes(root_dir)
# class_idx = index_classes(img_items)


# temp = dict()
# for imgname, classes, dirs in img_items:
#     img = '{}/{}'.format(dirs, imgname)
#     label = class_idx[classes]
#     transform = transforms.Compose([lambda img: Image.open(img).convert('L'),
#                               lambda img: img.resize((28,28)),
#                               lambda img: np.reshape(img, (28,28,1)),
#                               lambda img: np.transpose(img, [2,0,1]),
#                               lambda img: img/255.
#                               ])
#     img = transform(img)
#     if label in temp.keys():
#         temp[label].append(img)
#     else:
#         temp[label] = [img]
# print('begin to generate omniglot.npy')
# ## 移除标签信息，每个标签包含20个样本
# img_list = []
# for label, imgs in temp.items():
#     img_list.append(np.array(imgs))
# img_list = np.array(img_list).astype(np.float) # [[20 imgs],..., 1623 classes in total]
# print('data shape:{}'.format(img_list.shape)) # (1623, 20, 1, 28, 28)
# temp = []
# np.save(os.path.join(root_dir, 'omniglot.npy'), img_list)
# print('end.')

In [6]:
img_list = np.load(os.path.join(root_dir, 'omniglot.npy')) # (1623, 20, 1, 28, 28)
x_train = img_list[:1200]
x_test = img_list[1200:]
num_classes = img_list.shape[0]
datasets = {'train': x_train, 'test': x_test}

In [7]:
### 准备数据迭代器
n_way = 5
k_spt = 1  ## support data 的个数
k_query = 15 ## query data 的个数
imgsz = 28
resize = imgsz
task_num = 8
batch_size = task_num

indexes = {"train": 0, "test": 0}
datasets = {"train": x_train, "test": x_test}
print("DB: train", x_train.shape, "test", x_test.shape)


def load_data_cache(dataset):
    """
    Collects several batches data for N-shot learning
    :param dataset: [cls_num, 20, 84, 84, 1]
    :return: A list with [support_set_x, support_set_y, target_x, target_y] ready to be fed to our networks
    """
    #  take 5 way 1 shot as example: 5 * 1
    setsz = k_spt * n_way
    querysz = k_query * n_way
    data_cache = []

    # print('preload next 10 caches of batch_size of batch.')
    for sample in range(10):  # num of epochs

        x_spts, y_spts, x_qrys, y_qrys = [], [], [], []
        for i in range(batch_size):  # one batch means one set

            x_spt, y_spt, x_qry, y_qry = [], [], [], []
            selected_cls = np.random.choice(dataset.shape[0], n_way, replace =  False) 

            for j, cur_class in enumerate(selected_cls):

                selected_img = np.random.choice(20, k_spt + k_query, replace = False)

                # 构造support集和query集
                x_spt.append(dataset[cur_class][selected_img[:k_spt]])
                x_qry.append(dataset[cur_class][selected_img[k_spt:]])
                y_spt.append([j for _ in range(k_spt)])
                y_qry.append([j for _ in range(k_query)])

            # shuffle inside a batch
            perm = np.random.permutation(n_way * k_spt)
            x_spt = np.array(x_spt).reshape(n_way * k_spt, 1, resize, resize)[perm]
            y_spt = np.array(y_spt).reshape(n_way * k_spt)[perm]
            perm = np.random.permutation(n_way * k_query)
            x_qry = np.array(x_qry).reshape(n_way * k_query, 1, resize, resize)[perm]
            y_qry = np.array(y_qry).reshape(n_way * k_query)[perm]
 
            # append [sptsz, 1, 84, 84] => [batch_size, setsz, 1, 84, 84]
            x_spts.append(x_spt)
            y_spts.append(y_spt)
            x_qrys.append(x_qry)
            y_qrys.append(y_qry)

#         print(x_spts[0].shape)
        # [b, setsz = n_way * k_spt, 1, 84, 84]
        x_spts = np.array(x_spts).astype(np.float32).reshape(batch_size, setsz, 1, resize, resize)
        y_spts = np.array(y_spts).astype(np.int).reshape(batch_size, setsz)
        # [b, qrysz = n_way * k_query, 1, 84, 84]
        x_qrys = np.array(x_qrys).astype(np.float32).reshape(batch_size, querysz, 1, resize, resize)
        y_qrys = np.array(y_qrys).astype(np.int).reshape(batch_size, querysz)
#         print(x_qrys.shape)
        data_cache.append([x_spts, y_spts, x_qrys, y_qrys])

    return data_cache

datasets_cache = {"train": load_data_cache(x_train),  # current epoch data cached
                       "test": load_data_cache(x_test)}

def next(mode='train'):
    """
    Gets next batch from the dataset with name.
    :param mode: The name of the splitting (one of "train", "val", "test")
    :return:
    """
    # update cache if indexes is larger than len(data_cache)
    if indexes[mode] >= len(datasets_cache[mode]):
        indexes[mode] = 0
        datasets_cache[mode] = load_data_cache(datasets[mode])

    next_batch = datasets_cache[mode][indexes[mode]]
    indexes[mode] += 1

    return next_batch


DB: train (1200, 20, 1, 28, 28) test (423, 20, 1, 28, 28)


In [8]:
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy,copy
        

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.vars = nn.ParameterList()  ## 包含了所有需要被优化的tensor
        self.vars_bn = nn.ParameterList()
        
        # 第1个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 1, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        # 第1个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        running_mean = nn.Parameter(torch.zeros(64), requires_grad= False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad= False)
        self.vars_bn.extend([running_mean, running_var])
        
        # 第2个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        # 第2个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        running_mean = nn.Parameter(torch.zeros(64), requires_grad= False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad= False)
        self.vars_bn.extend([running_mean, running_var])
        
        # 第3个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        # 第3个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        running_mean = nn.Parameter(torch.zeros(64), requires_grad= False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad= False)
        self.vars_bn.extend([running_mean, running_var])
        
        # 第4个conv2d
        # in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2
        weight = nn.Parameter(torch.ones(64, 64, 3, 3))
        nn.init.kaiming_normal_(weight)
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        # 第4个BatchNorm层
        weight = nn.Parameter(torch.ones(64))
        bias = nn.Parameter(torch.zeros(64))
        self.vars.extend([weight,bias])
        
        running_mean = nn.Parameter(torch.zeros(64), requires_grad= False)
        running_var = nn.Parameter(torch.zeros(64), requires_grad= False)
        self.vars_bn.extend([running_mean, running_var])
        
        ##linear
        weight = nn.Parameter(torch.ones([5,64]))
        bias = nn.Parameter(torch.zeros(5))
        self.vars.extend([weight,bias])
        
#         self.conv = nn.Sequential(
#             nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
            
#             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
            
#             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2), 
            
#             nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (3,3), padding = 2, stride = 2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2), 
            
#             FlattenLayer(),
#             nn.Linear(64,5)
#         )        
 
        
    def forward(self, x, params = None, bn_training=True):
        '''
        :bn_training: set False to not update
        :return: 
        '''
        if params is None:
            params = self.vars
        
        weight, bias = params[0], params[1]  # 第1个CONV层
        x = F.conv2d(x, weight, bias, stride = 2, padding = 2)
        
        weight, bias = params[2], params[3]  # 第1个BN层
        running_mean, running_var = self.vars_bn[0], self.vars_bn[1]
        x = F.batch_norm(x, running_mean, running_var, weight=weight,bias =bias, training= bn_training)
        x = F.max_pool2d(x,kernel_size=2)  #第1个MAX_POOL层  
        x = F.relu(x, inplace = [True])  #第1个relu
        
        
        weight, bias = params[4], params[5]  # 第2个CONV层
        x = F.conv2d(x, weight, bias, stride = 2, padding = 2)
        
        weight, bias = params[6], params[7]  # 第2个BN层
        running_mean, running_var = self.vars_bn[2], self.vars_bn[3]
        x = F.batch_norm(x, running_mean, running_var, weight=weight,bias =bias, training= bn_training)
        x = F.max_pool2d(x,kernel_size=2)  #第2个MAX_POOL层   
        x = F.relu(x, inplace = [True])  #第2个relu
        
        weight, bias = params[8], params[9]  # 第3个CONV层
        x = F.conv2d(x, weight, bias, stride = 2, padding = 2)
        
        weight, bias = params[10], params[11]  # 第3个BN层
        running_mean, running_var = self.vars_bn[4], self.vars_bn[5]
        x = F.batch_norm(x, running_mean, running_var, weight=weight,bias =bias, training= bn_training)
        x = F.max_pool2d(x,kernel_size=2)  #第3个MAX_POOL层
        x = F.relu(x, inplace = [True])  #第3个relu
        
        weight, bias = params[12], params[13]  # 第4个CONV层
        x = F.conv2d(x, weight, bias, stride = 2, padding = 2)
        x = F.relu(x, inplace = [True])  #第4个relu
        weight, bias = params[14], params[15]  # 第4个BN层
        running_mean, running_var = self.vars_bn[6], self.vars_bn[7]
        x = F.batch_norm(x, running_mean, running_var, weight=weight,bias =bias, training= bn_training)
        x = F.max_pool2d(x,kernel_size=2)  #第4个MAX_POOL层
        
        
        x = x.view(x.size(0), -1) ## flatten
        weight, bias = params[16], params[17]  # linear
        x = F.linear(x, weight, bias)
        
        output = x
        
        return output
    
    
    def parameters(self):
        
        return self.vars


In [24]:
class MetaLearner(nn.Module):
    def __init__(self):
        super(MetaLearner, self).__init__()
        self.update_step = 5 ## task-level inner update steps
        self.update_step_test = 5  
        self.net = BaseNet()
        self.meta_lr = 2e-4
        self.base_lr = 4 * 1e-2
        self.inner_lr = 0.4
        self.outer_lr = 1e-2
        self.meta_optim = torch.optim.Adam(self.net.parameters(), lr = self.meta_lr)
        
    def forward(self,x_spt, y_spt, x_qry, y_qry):
        # 初始化
        task_num, ways, shots, h, w = x_spt.size()
        query_size = x_qry.size(1) # 75 = 15 * 5
        loss_list_qry = [0 for _ in range(self.update_step + 1)]
        correct_list = [0 for _ in range(self.update_step + 1)]
        
        for i in range(task_num):
            ## 第0步更新
            y_hat = self.net(x_spt[i], params = None, bn_training=True) # (ways * shots, ways)
            loss = F.cross_entropy(y_hat, y_spt[i]) 
            grad = torch.autograd.grad(loss, self.net.parameters())
            tuples = zip(grad, self.net.parameters()) ## 将梯度和参数\theta一一对应起来
            # fast_weights这一步相当于求了一个\theta - \alpha*\nabla(L)
            fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))
            # 在query集上测试，计算准确率
            # 这一步使用更新前的数据
            with torch.no_grad():
                y_hat = self.net(x_qry[i], self.net.parameters(), bn_training = True)
                loss_qry = F.cross_entropy(y_hat, y_qry[i])
                loss_list_qry[0] += loss_qry
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
                correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                correct_list[0] += correct
            
            # 使用更新后的数据在query集上测试。
            with torch.no_grad():
                y_hat = self.net(x_qry[i], fast_weights, bn_training = True)
                loss_qry = F.cross_entropy(y_hat, y_qry[i])
                loss_list_qry[1] += loss_qry
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
                correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                correct_list[1] += correct   
            
            for k in range(1, self.update_step):
                
                y_hat = self.net(x_spt[i], params = fast_weights, bn_training=True)
                loss = F.cross_entropy(y_hat, y_spt[i])
                grad = torch.autograd.grad(loss, fast_weights)
                tuples = zip(grad, fast_weights) 
                fast_weights = list(map(lambda p: p[1] - self.base_lr * p[0], tuples))
                    
                y_hat = self.net(x_qry[i], params = fast_weights, bn_training = True)
                loss_qry = F.cross_entropy(y_hat, y_qry[i])
                loss_list_qry[k+1] += loss_qry
                
                with torch.no_grad():
                    pred_qry = F.softmax(y_hat,dim=1).argmax(dim=1)
                    correct = torch.eq(pred_qry, y_qry[i]).sum().item()
                    correct_list[k+1] += correct
#         print('hello')
                
        loss_qry = loss_list_qry[-1] / task_num
        self.meta_optim.zero_grad() # 梯度清零
        loss_qry.backward()
        self.meta_optim.step()
        
        accs = np.array(correct_list) / (query_size * task_num)
        loss = np.array(loss_list_qry) / ( task_num)
        return accs,loss

    
    
    def finetunning(self, x_spt, y_spt, x_qry, y_qry):
        assert len(x_spt.shape) == 4
        
        query_size = x_qry.size(0)
        correct_list = [0 for _ in range(self.update_step_test + 1)]
        
        new_net = deepcopy(self.net)
        y_hat = new_net(x_spt)
        loss = F.cross_entropy(y_hat, y_spt)
        grad = torch.autograd.grad(loss, new_net.parameters())
        fast_weights = list(map(lambda p:p[1] - self.base_lr * p[0], zip(grad, new_net.parameters())))
        
        # 在query集上测试，计算准确率
        # 这一步使用更新前的数据
        with torch.no_grad():
            y_hat = new_net(x_qry,  params = new_net.parameters(), bn_training = True)
            pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
            correct = torch.eq(pred_qry, y_qry).sum().item()
            correct_list[0] += correct

        # 使用更新后的数据在query集上测试。
        with torch.no_grad():
            y_hat = new_net(x_qry, params = fast_weights, bn_training = True)
            pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)  # size = (75)
            correct = torch.eq(pred_qry, y_qry).sum().item()
            correct_list[1] += correct

        for k in range(1, self.update_step_test):
            y_hat = new_net(x_spt, params = fast_weights, bn_training=True)
            loss = F.cross_entropy(y_hat, y_spt)
            grad = torch.autograd.grad(loss, fast_weights)
            fast_weights = list(map(lambda p:p[1] - self.base_lr * p[0], zip(grad, fast_weights)))
            
            y_hat = new_net(x_qry, fast_weights, bn_training=True)
            
            with torch.no_grad():
                pred_qry = F.softmax(y_hat, dim=1).argmax(dim=1)
                correct = torch.eq(pred_qry, y_qry).sum().item()
                correct_list[k+1] += correct
                
        del new_net
        accs = np.array(correct_list) / query_size
        return accs
        

In [25]:
# net = torch.load('./trained_models/MTL-5000epochs.pt')

In [26]:
## omniglot

import time
device = torch.device('cuda')

meta = MetaLearner().to(device)

epochs = 60000
for step in range(epochs):
    start = time.time()
    x_spt, y_spt, x_qry, y_qry = next('train')
    x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device),\
                                 torch.from_numpy(y_spt).to(device),\
                                 torch.from_numpy(x_qry).to(device),\
                                 torch.from_numpy(y_qry).to(device)
    accs,loss = meta(x_spt, y_spt, x_qry, y_qry)
    end = time.time()
    if step % 100 == 0:
        print("epoch:" ,step)
        print(accs)
        print(loss)
        
    if step % 1000 == 0:
        accs = []
        for _ in range(1000//task_num):
            # db_train.next('test')
            x_spt, y_spt, x_qry, y_qry = next('test')
            x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device),\
                                         torch.from_numpy(y_spt).to(device),\
                                         torch.from_numpy(x_qry).to(device),\
                                         torch.from_numpy(y_qry).to(device)

            
            for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):
                test_acc = meta.finetunning(x_spt_one, y_spt_one, x_qry_one, y_qry_one)
                accs.append(test_acc)
        print('在mean process之前：',np.array(accs).shape)
        accs = np.array(accs).mean(axis=0).astype(np.float16)
        print('测试集准确率:',accs)
                                    

epoch: 0
[0.2        0.29333333 0.375      0.41333333 0.43       0.43666667]
[tensor(1.6094, device='cuda:0') tensor(1.5602, device='cuda:0')
 tensor(1.5179, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.4858, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.4404, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.4106, device='cuda:0', grad_fn=<DivBackward0>)]
在mean process之前： (1000, 6)
测试集准确率: [0.2009 0.2825 0.3257 0.3396 0.3528 0.36  ]
epoch: 100
[0.2        0.515      0.55666667 0.59166667 0.605      0.6       ]
[tensor(1.6131, device='cuda:0') tensor(1.3106, device='cuda:0')
 tensor(1.1762, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.1033, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.0616, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(1.0450, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 200
[0.29166667 0.53833333 0.61333333 0.62833333 0.63833333 0.645     ]
[tensor(1.5989, device='cuda:0') tensor(1.1745, device='cuda:0')
 tensor(1.0416, device='cuda:0', 

epoch: 2200
[0.2        0.75       0.86       0.905      0.89833333 0.89666667]
[tensor(4.6417, device='cuda:0') tensor(0.8151, device='cuda:0')
 tensor(0.5254, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.4472, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.4210, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.4045, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 2300
[0.2        0.85166667 0.905      0.905      0.90833333 0.90833333]
[tensor(5.4049, device='cuda:0') tensor(0.6904, device='cuda:0')
 tensor(0.4569, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3960, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3671, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3483, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 2400
[0.2        0.81       0.85166667 0.86833333 0.87       0.87      ]
[tensor(6.1629, device='cuda:0') tensor(0.6922, device='cuda:0')
 tensor(0.5321, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.4724, device='cuda:0', grad_fn=<DivBackw

epoch: 4400
[0.2        0.88333333 0.89833333 0.895      0.89666667 0.895     ]
[tensor(10.6987, device='cuda:0') tensor(0.4104, device='cuda:0')
 tensor(0.3662, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3510, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3445, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3399, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 4500
[0.2        0.87333333 0.88833333 0.89166667 0.91       0.91166667]
[tensor(10.5549, device='cuda:0') tensor(0.3994, device='cuda:0')
 tensor(0.3573, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.5227, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3035, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2968, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 4600
[0.2        0.84166667 0.85833333 0.865      0.87333333 0.87      ]
[tensor(10.8610, device='cuda:0') tensor(0.4529, device='cuda:0')
 tensor(0.3946, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.3796, device='cuda:0', grad_fn=<DivBa

epoch: 6600
[0.2        0.92       0.91333333 0.925      0.92166667 0.92333333]
[tensor(13.0907, device='cuda:0') tensor(0.2292, device='cuda:0')
 tensor(0.2271, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2065, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2063, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2055, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 6700
[0.2        0.93       0.94166667 0.94333333 0.94333333 0.945     ]
[tensor(13.2028, device='cuda:0') tensor(0.2456, device='cuda:0')
 tensor(0.2240, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2175, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2138, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.2111, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 6800
[0.2        0.92       0.94       0.94166667 0.945      0.945     ]
[tensor(13.0057, device='cuda:0') tensor(0.2091, device='cuda:0')
 tensor(0.1738, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1665, device='cuda:0', grad_fn=<DivBa

epoch: 8800
[0.2        0.90833333 0.935      0.935      0.935      0.93666667]
[tensor(13.8422, device='cuda:0') tensor(0.2537, device='cuda:0')
 tensor(0.1910, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1870, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1840, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1817, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 8900
[0.2        0.915      0.89833333 0.92666667 0.92666667 0.93      ]
[tensor(14.1322, device='cuda:0') tensor(0.2398, device='cuda:0')
 tensor(0.2649, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1960, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1940, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1925, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 9000
[0.2        0.95666667 0.95833333 0.95833333 0.95833333 0.95833333]
[tensor(14.0523, device='cuda:0') tensor(0.1334, device='cuda:0')
 tensor(0.1305, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.1292, device='cuda:0', grad_fn=<DivBa

...

epoch: 50700
[0.2        0.99333333 0.99333333 0.99333333 0.99333333 0.99333333]
[tensor(9.9677, device='cuda:0') tensor(0.0186, device='cuda:0')
 tensor(0.0186, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0186, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0186, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0186, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 50800
[0.2  0.98 0.98 0.98 0.98 0.98]
[tensor(10.0960, device='cuda:0') tensor(0.0973, device='cuda:0')
 tensor(0.0973, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0973, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0973, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0973, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 50900
[0.2  0.99 0.99 0.99 0.99 0.99]
[tensor(10.0286, device='cuda:0') tensor(0.0214, device='cuda:0')
 tensor(0.0214, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0214, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0214, device='cuda:0', grad_fn=<DivBackward0>)
 te

epoch: 53000
[0.2        0.99833333 0.99833333 0.99833333 0.99833333 0.99833333]
[tensor(10.0213, device='cuda:0') tensor(0.0140, device='cuda:0')
 tensor(0.0141, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0141, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0142, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0143, device='cuda:0', grad_fn=<DivBackward0>)]
在mean process之前： (1000, 6)
测试集准确率: [0.2    0.919  0.9194 0.9194 0.9194 0.9194]
epoch: 53100
[0.2        0.99833333 0.99833333 0.99833333 0.99833333 0.99833333]
[tensor(9.9904, device='cuda:0') tensor(0.0055, device='cuda:0')
 tensor(0.0055, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0055, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0055, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0055, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 53200
[0.2        0.99666667 0.99666667 0.99666667 0.99666667 0.99666667]
[tensor(10.0705, device='cuda:0') tensor(0.0091, device='cuda:0')
 tensor(0.0091, device=

epoch: 55200
[0.2        0.99666667 0.99666667 0.99666667 0.99666667 0.99666667]
[tensor(10.2133, device='cuda:0') tensor(0.0062, device='cuda:0')
 tensor(0.0062, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0062, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0062, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0062, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 55300
[0.2        0.99666667 0.99666667 0.99666667 0.99666667 0.99666667]
[tensor(10.2065, device='cuda:0') tensor(0.0116, device='cuda:0')
 tensor(0.0116, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0116, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0116, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0116, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 55400
[0.2        0.99166667 0.99166667 0.99166667 0.99166667 0.99166667]
[tensor(10.2829, device='cuda:0') tensor(0.0246, device='cuda:0')
 tensor(0.0243, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0241, device='cuda:0', grad_fn=<Di

epoch: 57500
[0.2        0.98666667 0.98666667 0.98666667 0.98666667 0.98666667]
[tensor(9.9706, device='cuda:0') tensor(0.0291, device='cuda:0')
 tensor(0.0291, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0291, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0291, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0291, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 57600
[0.2  0.99 0.99 0.99 0.99 0.99]
[tensor(9.8868, device='cuda:0') tensor(0.0342, device='cuda:0')
 tensor(0.0342, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0342, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0342, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0342, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 57700
[0.2        0.99666667 0.99666667 0.99666667 0.99666667 0.99666667]
[tensor(9.8927, device='cuda:0') tensor(0.0108, device='cuda:0')
 tensor(0.0108, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0107, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0107, device='cu

epoch: 59800
[0.2   0.995 0.995 0.995 0.995 0.995]
[tensor(10.1850, device='cuda:0') tensor(0.0165, device='cuda:0')
 tensor(0.0165, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0165, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0165, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0165, device='cuda:0', grad_fn=<DivBackward0>)]
epoch: 59900
[0.2        0.98833333 0.98833333 0.98833333 0.98833333 0.98833333]
[tensor(10.2750, device='cuda:0') tensor(0.0321, device='cuda:0')
 tensor(0.0321, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0320, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0320, device='cuda:0', grad_fn=<DivBackward0>)
 tensor(0.0319, device='cuda:0', grad_fn=<DivBackward0>)]
